In [1]:
from trans_tools import *
from IrisBackendv3.codec.payload import CommandPayload
from IrisBackendv3.codec.packet import IrisCommonPacket
from IrisBackendv3.codec.metadata import DataPathway, DataSource
from IrisBackendv3.codec.magic import Magic
from IrisBackendv3.utils.basic import bytearray_to_spaced_hex as hexstr
# connect_serial(device = '/dev/ttyUSB0')


search here: ../FlightSoftware/fprime/./CubeRover/Top/CubeRoverTopologyAppAi.xml


In [2]:
settings['SAVE_FILE_PREFIX'] = 'iris_pims3_prep0' # this the prefix on all log files. make it something unique.
load_cache()

In [3]:
list_commands_wd()
standards.print_overview()


	0.	'WatchDogInterface_ResetSpecific', args: [reset_value: enum/*int32*/]

			Available values for `reset_value`:

				'NO_RESET' or 0 or 0x0

				'RESET_HERCULES' or 1 or 0x1

				'HERCULES_POWER_ON' or 2 or 0x2

				'HERCULES_POWER_OFF' or 3 or 0x3

				'RESET_RADIO' or 4 or 0x4

				'RADIO_POWER_ON' or 5 or 0x5

				'RADIO_POWER_OFF' or 6 or 0x6

				'RESET_FPGA' or 7 or 0x7

				'FPGA_POWER_ON' or 8 or 0x8

				'FPGA_POWER_OFF' or 9 or 0x9

				'RESET_MOTOR_1' or 10 or 0xa

				'RESET_MOTOR_2' or 11 or 0xb

				'RESET_MOTOR_3' or 12 or 0xc

				'RESET_MOTOR_4' or 13 or 0xd

				'RESET_ALL_MOTORS' or 14 or 0xe

				'ALL_MOTORS_ON' or 15 or 0xf

				'ALL_MOTORS_OFF' or 16 or 0x10

				'RESET_EN_3_3' or 17 or 0x11

				'EN_3_3_POWER_ON' or 18 or 0x12

				'EN_3_3_POWER_OFF' or 19 or 0x13

				'RESET_24_EN' or 20 or 0x14

				'EN_24_ON' or 21 or 0x15

				'EN_24_OFF' or 22 or 0x16

				'HDRM_OFF' or 24 or 0x18

				'FPGA_CAM_0' or 25 or 0x19

				'FPGA_CAM_1' or 26 or 0x1a

				'BATTE

In [4]:
seq_num = 0x00

In [5]:
seq_num += 1
# seq_num = 0x05

pathway = DataPathway.WIRED
source = DataSource.GENERATED

magic = Magic.WATCHDOG_COMMAND
command_name = 'WatchDogInterface_SwitchToServiceMode'
kwargs = dict(confirm='CONFIRM_SERVICE')
# command_name = 'WatchDogInterface_PrepareForDeployment'
# kwargs = dict(confirm='CONFIRM_PREP')
# command_name = 'WatchDogInterface_DisengageFromLander'
# kwargs = dict(confirm='CONFIRM_DEPLOY')

module, command = standards.global_command_lookup(command_name)
payloads = PayloadCollection(
    CommandPayload=[
        CommandPayload(
            pathway=pathway,
            source=source,
            magic=magic,
            module_id=module.ID,
            command_id=command.ID,
            args=kwargs
        )
    ],
    TelemetryPayload=[],
    EventPayload=[],
    FileBlockPayload=[]
)
packet = IrisCommonPacket(
    seq_num = seq_num,
    payloads = payloads,
    pathway = pathway,
    source = source
).encode()
print(hexstr(packet))

# send_wifi(packet, ip="192.168.1.2", port=8080)
# yamcs_payload = dict(
#     assignment = [
#         {
#             "name": "tc-binary",
#             "value": hexstr(packet).replace(' ','')
#         }
#     ]
# )
yamcs_payload = {
    'tc-binary': hexstr(packet).replace(' ','')
}
yamcs_payload

01 0a 00 d4 ee ff 00 c0 ec 10 77 00 00 00


{'tc-binary': '010a00d4eeff00c0ec1077000000'}

In [13]:
# import requests
# url = '192.168.10.2:8090/api/processors/Astrobotic-M1/realtime/commands/Astrobotic-M1/WatchDogInterface_SwitchToServiceMode'
# auth = ('iris', 'justanexample')
# headers = {'Content-Type': 'application/json'}
HOSTNAME = "192.168.10.105"
PORT = "8090"
URI = "{0}:{1}".format(HOSTNAME, PORT)

YAMCS_INSTANCE = 'Astrobotic-M1'
YAMCS_PROCESSOR = 'realtime'

from yamcs.client import YamcsClient
from yamcs.core.auth import Credentials as YamcsCredentials

# Sign in with username and password defined at top of file
credentials = YamcsCredentials(username='iris', password='justanexample')

# Create a yamcs client that connects with our server.
# Currently connects to localhost but will change later
client = YamcsClient(URI, credentials=credentials)

# Send a command:
yamcs_cmd_name = command_name + pathway.yamcs_suffix
processor = client.get_processor(instance=YAMCS_INSTANCE, processor=YAMCS_PROCESSOR)
status = processor.issue_command(yamcs_cmd_name, args=yamcs_payload)
print("Command status: ", status.__dict__)

ConnectionError: HTTPConnectionPool(host='localhost', port=8090): Max retries exceeded with url: /auth/token (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x13c859250>: Failed to establish a new connection: [Errno 61] Connection refused'))